In [ ]:
#pip install open3d

In [ ]:
#pip install -U matplotlib

In [ ]:
#pip install --upgrade setuptools

In [ ]:
#pip install imageio

In [ ]:
#pip install -U pandas

In [ ]:
#pip install pyequilib

In [ ]:
#pip install seaborn

In [1]:
import imageio as iio
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from equilib import Equi2Pers

C:\Users\Fan_Z\anaconda3\envs\tf-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Fan_Z\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Fan_Z\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### create point cloud from panorama

In [8]:
pnt = []  # 3D point list x0, y0, z0, x1, y1, z1, ...
clr = []
remove_x = []
remove_y = []
from PIL import Image

x, y, xs, ys = 0, 0, 0, 0  # texture position and size
a, b, r, da, db = 0.0, 0.0, 0.0, 0.0, 0.0  # spherical position and angle steps
xx, yy, zz = 0.0, 0.0, 0.0  # 3D point
    
depth_img = Image.open('../Project-demo/sample_data/depth_predict_4.png')
color_img = Image.open('../Project-demo/sample_data/test_img_4.jpg')

xs, ys = depth_img.size

# 180x90 deg
da = 2.0 * 3.14 / (xs - 1)
db = 1 * 3.14 / (ys - 1)
b = -0.5 * 3.14
depth_img_array = np.asarray(depth_img)
depth_max = depth_img_array.max()


b_spilt_0 = 0
b_spilt = 1/(ys - 1)

# Process all its pixels
pnt.clear()
for y in range(ys):
    #from negative 90 to positive 90 vertically 
    b += db
    b_spilt_0 += b_spilt
    
    for x in range(xs):
        # from 0 to 360 horizontally 
        a += da

        # Pixel access
        r1 = depth_max - depth_img.getpixel((x, y))  # obtain intensity from texture <0..255>
        
        # remove the points within the central area
            
        r2 = r1/float(depth_max)  # normalize to <0..1>

        # Convert to 3D
        xx = 2 * r2 * np.cos(a) * np.cos(b)
        yy = 2 * r2 * np.sin(a) * np.cos(b)
        zz = (2*r2) ** 2 *np.sin(b)
           
        c = color_img.getpixel((x, y))
        c0 = c[0]/255.0
        c1 = c[1]/255.0
        c2 = c[2]/255.0
        
        #pnt.append([xx,yy,zz])
        #clr.append([c0,c1,c2])
        
        if r1>20 and zz <0.2:
            pnt.append([xx,yy,zz])
            clr.append([c0,c1,c2])
        
        

In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pnt)
pcd.colors = o3d.utility.Vector3dVector(np.array(clr))

o3d.visualization.draw_geometries([pcd])

#### create point cloud from splited image

In [10]:
import PIL.Image as pil
path = '../Project-demo/sample_data/depth_predict_4.png'
depth_raw  = pil.open(path).convert('RGB')
width,height = depth_raw.size

FOV = 90
f = 0.5 * width * 1 / np.tan(0.5 * FOV / 180.0 * np.pi)
cx = (width - 1) / 2.0
cy = (height - 1) / 2.0
fx = f * width
fy = f * height

phc = o3d.camera.PinholeCameraIntrinsic()
phc.set_intrinsics(width, height, fx, fy, cx, cy)
phc.intrinsic_matrix

array([[2.097152e+06, 0.000000e+00, 1.023500e+03],
       [0.000000e+00, 1.048576e+06, 5.115000e+02],
       [0.000000e+00, 0.000000e+00, 1.000000e+00]])

In [17]:
# 
for i in range(2):
    
    depth_file = '../Project-demo/sample_data/depth_predict_4_spilt_{}.png'.format(str(i))
    rgb_file = '../Project-demo/sample_data/test_img_4_spilt_{}.png'.format(str(i))
    
    depth_image = o3d.io.read_image(depth_file)
    rgb_image = o3d.io.read_image(rgb_file)
    
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb_image,depth_image)
    print(rgbd_image)



RGBDImage of size 
Color image : 720x720, with 1 channels.
Depth image : 720x720, with 1 channels.
Use numpy.asarray to access buffer data.
RGBDImage of size 
Color image : 720x720, with 1 channels.
Depth image : 720x720, with 1 channels.
Use numpy.asarray to access buffer data.


In [19]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
